In [1]:
import random
import os
import numpy as np
import pandas as pd
from datetime import datetime
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)
np.random.seed(42)

# ==========================================
# 1. 데이터 로드
# ==========================================

train_df = pd.read_csv('../Data/train.csv')
test_df = pd.read_csv('../Data/test.csv')
target = '임신 성공 여부'

# ==========================================
# 2. 파생변수 생성
# ==========================================

def derive_features(df):

    # 1. 수치형 변환 및 클리닝 (정규표현식 활용)
    count_cols = [
        '총 시술 횟수', '클리닉 내 총 시술 횟수', '총 임신 횟수', '총 출산 횟수',
        'IVF 시술 횟수', 'DI 시술 횟수', 'IVF 임신 횟수', 'DI 임신 횟수',
        'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수', '이식된 배아 수', 
        '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '저장된 배아 수'
    ]
    
    for col in count_cols:
        if df[col].dtype == 'object':
            # '회', ' 이상' 등을 제거하고 공백 정리 후 숫자형으로 변환
            df[col] = df[col].astype(str).str.replace(r'회| 이상', '', regex=True).str.strip()
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

    target_cols = [
            '총 생성 배아 수', '기증자 정자와 혼합된 난자 수', '파트너 정자와 혼합된 난자 수',
            '미세주입된 난자 수', '혼합된 난자 수', '저장된 신선 난자 수', '해동 난자 수',
            '해동된 배아 수', '미세주입 후 저장된 배아 수', '저장된 배아 수',
            '미세주입 배아 이식 수', '이식된 배아 수', '미세주입에서 생성된 배아 수',
            '수집된 신선 난자 수', '단일 배아 이식 여부', '착상 전 유전 진단 사용 여부', '대리모 여부',
            '기증 배아 사용 여부', '신선 배아 사용 여부', '동결 배아 사용 여부'
        ]

    # 시술 유형이 DI인 행에 대해서만 지정된 컬럼들의 NaN을 0으로 채움
    df.loc[df['특정 시술 유형'] == 'DI', target_cols] = df.loc[df['특정 시술 유형'] == 'DI', target_cols].fillna(0)


    # 2. 연령 관련 변수 생성
    age_order = {
        '만18-34세': 1, '만35-37세': 2, '만38-39세': 3, 
        '만40-42세': 4, '만43-44세': 5, '만45-50세': 6, '알 수 없음': 7
    }
    df['시술 당시 나이'] = df['시술 당시 나이'].map(age_order).fillna(7)
    
    # 파생 변수 계산 (Zero Division 방지를 위해 1e-6 활용)
    epsilon = 1e-6
    df['나이 이식배아 비율'] = df['시술 당시 나이'] * df['이식된 배아 수']
    df['연령별 배아 효율'] = df['이식된 배아 수'] / (df['시술 당시 나이'] + epsilon)
    df['이식 배아 효율'] = df['이식된 배아 수'] * df['연령별 배아 효율']
    df['배아 발달 기간'] = df['배아 이식 경과일'] - df['난자 혼합 경과일']
    df['이식 비중'] = df['이식된 배아 수'] / (df['이식된 배아 수'] + df['저장된 배아 수'] + epsilon)

    # 3. 난자 및 배아 효율성 변수
    oocyte_cols = ['수집된 신선 난자 수', '혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '해동 난자 수']
    df['총 난자 수'] = df[oocyte_cols].fillna(0).sum(axis=1)
    
    # 0으로 나누기 방지를 위해 총 난자 수가 0인 경우 0으로 처리하거나 epsilon 추가
    df['배아 손실률'] = (df['총 난자 수'] - df['총 생성 배아 수']) / (df['총 난자 수'] + epsilon)
    df['배아 생성 효율'] = df['저장된 배아 수'] / (df['저장된 신선 난자 수'] + epsilon)
    df['수정 효율'] = df['총 생성 배아 수'] / (df['혼합된 난자 수'] + epsilon)
    df['선별 효율'] = df['저장된 배아 수'] / (df['총 생성 배아 수'] + epsilon)

    # 4. 시술 유형 분류 (Vectorized 방식 권장)
    def clean_treatment(val):
        val = str(val).upper()
        if 'ICSI' in val: return 'ICSI'
        if 'IVF' in val: return 'IVF'
        if 'IUI' in val: return 'IUI'
        return 'Other'

    df['특정 시술 유형'] = df['특정 시술 유형'].apply(clean_treatment)

    # 5. 불임 원인 점수 계산
    female_cols = [
        '불임 원인 - 여성 요인', '불임 원인 - 난관 질환', '불임 원인 - 배란 장애', 
        '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '여성 주 불임 원인', 
        '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인'
    ]
    male_cols = [
        '불임 원인 - 남성 요인', '불임 원인 - 정자 농도', '불임 원인 - 정자 운동성', 
        '불임 원인 - 정자 형태', '불임 원인 - 정자 면역학적 요인', '남성 주 불임 원인', 
        '남성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인'
    ]
    
    df['여성_결함_점수'] = df[female_cols].sum(axis=1)
    df['남성_결함_점수'] = df[male_cols].sum(axis=1)

    # 6. 배아 생성 주요 이유 단순화
    def clean_reason(x):
        if pd.isna(x): return 'Unknown'
        x = str(x)
        if '시술용' in x: return 'Treatment'
        if '기증' in x: return 'Donation'
        if '저장' in x: return 'Storage'
        return 'Other'
    
    df['배아 생성 주요 이유'] = df['배아 생성 주요 이유'].apply(clean_reason)
    
    # 7. 불필요한 컬럼 삭제
    drop_cols = [
    'ID', 
    '시술 시기 코드',
    '저장된 배아 수', 
    '착상 전 유전 검사 사용 여부', 
    'PGS 시술 여부', 
    'DI 출산 횟수', 
    '대리모 여부', 
    '시술 유형',
    ]
    df.drop(columns=drop_cols + female_cols + male_cols, errors='ignore', inplace=True)
    
    return df

train_df = derive_features(train_df)
test_df = derive_features(test_df)

# ==========================================
# 3. 모델 학습 설정
# ==========================================

predictor = TabularPredictor(
    label=target, 
    eval_metric='roc_auc',
    path='ag_models_out_v14',
).fit(
    train_data=train_df,
    time_limit=10800,
    presets='high_quality',
    num_stack_levels=1,
    num_bag_folds=7,
    num_bag_sets=1,
    refit_full=True,
    dynamic_stacking=False,
    included_model_types=['GBM', 'CAT', 'XGB'],
    ag_args_fit={'num_gpus': 1},
    save_space=True,
    set_best_to_refit_full=True
)

# ==========================================
# 4. 예측 (Test Data 활용) - 최종 결과를 확률로 출력 (Positive 클래스에 대한 확률만 추출)
# ==========================================

pred_probs = predictor.predict_proba(test_df)
final_probs = pred_probs.iloc[:, 1]

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.11.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 25.2.0: Tue Nov 18 21:09:40 PST 2025; root:xnu-12377.61.12~1/RELEASE_ARM64_T6000
CPU Count:          10
Pytorch Version:    2.9.1
CUDA Version:       CUDA is not available
GPU Count:          WARNING: Exception was raised when calculating GPU count (AssertionError)
Memory Avail:       7.72 GB / 16.00 GB (48.2%)
Disk Space Avail:   166.36 GB / 460.43 GB (36.1%)
Presets specified: ['high_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=7, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~7x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this

In [2]:
# --- 리더보드 출력 ---
lb = predictor.leaderboard(train_df, silent=True)
display(lb.head())

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost_r33_BAG_L1_FULL,0.782093,NaN,roc_auc,0.960856,NaN,4.258863,0.960856,NaN,4.258863,1,True,24
1,XGBoost_r34_BAG_L1_FULL,0.780624,NaN,roc_auc,0.937105,NaN,2.931376,0.937105,NaN,2.931376,1,True,32
2,XGBoost_r98_BAG_L1_FULL,0.767028,NaN,roc_auc,0.778849,NaN,11.921486,0.778849,NaN,11.921486,1,True,27
3,XGBoost_r33_BAG_L2_FULL,0.755314,NaN,roc_auc,8.301901,NaN,42.793716,1.011382,NaN,4.298037,2,True,35
4,XGBoost_r34_BAG_L2_FULL,0.755244,NaN,roc_auc,8.239201,NaN,41.626680,0.948682,NaN,3.131001,2,True,43


In [3]:
# --- 제출 파일 생성 ---
submission = pd.read_csv('../Data/sample_submission.csv')
submission['probability'] = final_probs.values

# 현재 시간 가져오기 (예: 0206_1031)
now = datetime.now().strftime('%m%d_%H%M')
file_name = f"{now}_submission.csv"
submission.to_csv(file_name, index=False)

print(f"학습 및 예측이 완료되었습니다. 결과가 {file_name}에 저장되었습니다.")

학습 및 예측이 완료되었습니다. 결과가 0209_2344_submission.csv에 저장되었습니다.


In [5]:
# --- 피처 중요도 ---
fi = predictor.feature_importance(data=train_df.sample(n=min(5000, len(train_df)), random_state=42))
fi.to_excel('fi14.xlsx')

Computing feature importance via permutation shuffling for 56 features using 5000 rows with 5 shuffle sets...
	41.77s	= Expected runtime (8.35s per shuffle set)
	27.09s	= Actual runtime (Completed 5 of 5 shuffle sets)
